# About amplifiers
Hanspeter Schmid, FHNW/ISE, 4.5.21

In [1]:
%matplotlib notebook
import sympy as sp
import numpy as np
import math
import scipy as sc
import scipy.signal
import matplotlib.pyplot as plt
import hanspitools as ht
def niceT(T,s): # This is a shortcut for a functionality I use very often
    return ht.mani.numden(lambda p: ht.mani.nicepoly(p,s),T)
def keeper(f,v):
    return sp.limit(f/v,v,sp.oo)*v

In [2]:
import sympy as sp
Delta = sp.symbols('Delta')
L1 = sp.symbols('L1')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
A,G2,Y1 = sp.symbols('A,G2,Y1')

loops = [(L1, -A*G2/Y1)]
determinant = [(Delta, 1 - L1)]
denominator = [(T_den, Delta)]

P1,D1 = sp.symbols('P1,D1')
paths = [(P1, A), (D1, 1)]
numerator = [(T_num, D1*P1)]

transfer_function = [(T_io, T_num/T_den)]
T=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T)

A*Y1/(A*G2 + Y1)

In [3]:
G1,s,omega1 = sp.symbols('G1,s,omega1')
dpa = [(Y1, G1+G2)]
amp = [(A,  omega1/s)]

In [4]:
T.subs(dpa)

A*(G1 + G2)/(A*G2 + G1 + G2)

In [5]:
T_w = T.subs(dpa).subs(amp).simplify()
T_w

omega1*(G1 + G2)/(G2*omega1 + s*(G1 + G2))

DC gain

In [6]:
T_DC = T_w.subs(s,0)
T_DC

(G1 + G2)/G2

BW of 1st-order lowpass filter: frequency of the pole

In [7]:
[n,d] = sp.fraction(T_w)
d

G2*omega1 + s*(G1 + G2)

In [8]:
T_BW = -sp.solve(d,s)[0]
T_BW

G2*omega1/(G1 + G2)

Gain-bandwidth product

In [9]:
T_DC*T_BW

omega1

Negative amplifier ($V_4 \to V_o$)

In [10]:
P1,D1 = sp.symbols('P1,D1')
paths = [(P1, -A*G1/Y1), (D1, 1)]
numerator = [(T_num, D1*P1)]

transfer_function = [(T_io, T_num/T_den)]
T_m=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(T_m)

-A*G1/(A*G2 + Y1)

In [11]:
T_m_w = T_m.subs(dpa).subs(amp).simplify()
T_m_w

-G1*omega1/(G2*omega1 + s*(G1 + G2))

In [12]:
T_m_w.subs(s,0)

-G1/G2

In [13]:
T_m_w.subs(s,0)*T_BW

-G1*omega1/(G1 + G2)

Loop gain

In [14]:
-L1.subs(loops).subs(dpa)

A*G2/(G1 + G2)

## OpAmp with output resistance

In [15]:
import sympy as sp
Delta = sp.symbols('Delta')
L1 = sp.symbols('L1')
T_num = sp.symbols('T_num')
T_den = sp.symbols('T_den')
T_io = sp.symbols('T_io')
G2,Y1,A,G0,Y3 = sp.symbols('G2,Y1,A,G0,Y3')

loops = [(L1, -A*G0*G2/(Y1*Y3))]
determinant = [(Delta, 1 - L1)]
denominator = [(T_den, Delta)]

P1,D1 = sp.symbols('P1,D1')
paths = [(P1, 1/Y3), (D1, 1)]
numerator = [(T_num, D1*P1)]

transfer_function = [(T_io, T_num/T_den)]
Z_o=T_io.subs(transfer_function).subs(numerator).subs(denominator).subs(determinant).subs(paths).subs(loops).simplify()
display(Z_o)

Y1/(A*G0*G2 + Y1*Y3)

In [16]:
G1,s,omega1 = sp.symbols('G1,s,omega1')
dpa = [(Y1, G1+G2), (Y3, G0+G2)]
amp = [(A,  omega1/s)]

In [17]:
Z_o.subs(dpa).subs(amp).simplify()

s*(G1 + G2)/(G0*G2*omega1 + s*(G0 + G2)*(G1 + G2))

Snubber network

In [18]:
G1,s,omega1,Y_L = sp.symbols('G1,s,omega1,Y_L')
dpa = [(Y1, G1+G2), (Y3, G0+G2+Y_L)]
amp = [(A,  omega1/s)]

In [19]:
Z_o.subs(dpa).subs(amp).simplify()

s*(G1 + G2)/(G0*G2*omega1 + s*(G1 + G2)*(G0 + G2 + Y_L))

In [20]:
C_L,C_s,G_s = sp.symbols('C_L,C_s,G_s')
eq_YL = [(Y_L,  s*C_L + 1/(1/G_s+1/(s*C_s)) )]

In [21]:
Z_snub = Z_o.subs(dpa).subs(amp).subs(eq_YL).simplify()
Z_snub

s*(G1 + G2)*(C_s*s + G_s)/(G0*G2*omega1*(C_s*s + G_s) + s*(G1 + G2)*(C_s*G_s*s + (C_s*s + G_s)*(C_L*s + G0 + G2)))

In [22]:
niceT(Z_snub,s)

(C_s*s**2*(G1 + G2) + G_s*s*(G1 + G2))/(C_L*C_s*s**3*(G1 + G2) + G0*G2*G_s*omega1 + s**2*(G1 + G2)*(C_L*G_s + C_s*G0 + C_s*G2 + C_s*G_s) + s*(C_s*G0*G2*omega1 + G0*G1*G_s + G0*G2*G_s + G1*G2*G_s + G2**2*G_s))